# Import Required Libraries
Import necessary libraries such as pandas, numpy, scikit-learn, matplotlib, and seaborn for clustering and visualization.

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set(style="whitegrid")

# Load Dataset with Embeddings
Load the previously generated dataset containing embeddings into a pandas DataFrame.

In [ ]:
# Load dataset with embeddings
df = pd.read_json('output_folder\\dataset_with_2d_embeddings.json', lines=True)

# Prepare Embeddings for Clustering
Extract embeddings from the DataFrame and convert them into a suitable format (e.g., numpy array) for clustering.

In [ ]:
# Extract embeddings into a numpy array
embeddings = np.array(df['embeddings_2d'].tolist(), dtype=np.float64)

# Check the shape of the embeddings array
print(f"Shape of DataFrame: {df.shape}")
print(f"Shape of Embeddings: {embeddings.shape}")
print(f"First few elements of Embeddings: {embeddings[:5]}")

# Determine Optimal Number of Clusters
Use methods such as the Elbow method or Silhouette analysis to determine the optimal number of clusters.

In [ ]:
# Determine optimal number of clusters using Elbow method
inertia = []
k_range = range(2, 15)

# Calculate inertia for different values of k
# Innertia is the sum of squared distances to the nearest cluster center
# It helps to find the optimal number of clusters by looking for the "elbow" point in the plot
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(embeddings)
    inertia.append(kmeans.inertia_)

# Plot the Elbow curve
# The Elbow method is a heuristic used in determining the number of clusters in a dataset.
# It involves plotting the explained variation as a function of the number of clusters and selecting the "elbow" of the curve as the number of clusters to use.
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.show()

# Determine optimal number of clusters using Silhouette analysis
# Silhouette analysis is a method of interpretation and validation of consistency within clusters of data.
# The silhouette score measures how similar an object is to its own cluster compared to other clusters.
silhouette_scores = []
k_range = range(2, 8)  # smaller range for efficiency

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(embeddings)
    score = silhouette_score(embeddings, labels)
    silhouette_scores.append(score)

# Plot Silhouette scores
plt.figure(figsize=(8, 5))
plt.plot(k_range, silhouette_scores, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis for Optimal k')
plt.show()

# Perform Clustering
Apply clustering algorithms such as KMeans or DBSCAN to group embeddings into clusters.

In [ ]:
# Choose optimal number of clusters based on previous analysis
optimal_k = 3

# Perform KMeans clustering
kmeans = KMeans(
    n_clusters=optimal_k,
    init='k-means++',
    n_init=20,
    max_iter=300,
    tol=1e-4,
    random_state=42,
    verbose=1
)

# Fit the model to the embeddings
df['cluster'] = kmeans.fit_predict(embeddings)

# Inspect cluster distribution
print(df['cluster'].value_counts())

# Visualize Clusters
Visualize the resulting clusters using dimensionality reduction techniques like PCA or t-SNE.

In [ ]:
# Reduce dimensionality using PCA
#pca = PCA(n_components=2, random_state=42) 
#reduced_embeddings = pca.fit_transform(embeddings) 

# Plot clusters
plt.figure(figsize=(10, 7))
sns.scatterplot(x=embeddings[:,0], y=embeddings[:,1], hue=df['cluster'], palette='viridis', s=50) 
plt.title('Clusters Visualization using PCA')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(title='Cluster')
plt.show()

# Save Clustered Data
Save the DataFrame with cluster labels to a new file for further analysis.

In [ ]:
# Save clustered DataFrame to JSON file
df.to_json('output_folder\\clustered_dataset.json', orient='records', lines=True)

print("Clustered dataset has been successfully saved to 'clustered_dataset.json'.")